In [1]:
import pandas as pd
import numpy as np
import re
import warnings

# Suppress DataFrame fragmentation warnings
warnings.filterwarnings('ignore', message='DataFrame is highly fragmented')

In [2]:
df_gdrive = pd.read_excel('/Users/luke.thompson/Google Drive/My Drive/NOAA/okex/metadata/Okeanos_eDNA_Sample_Tracking_GDrive_LRT.xlsx')
df_geome = pd.read_excel('/Users/luke.thompson/Google Drive/My Drive/NOAA/okex/metadata/geome-fims-output-csv-okeanos/Events_output_LRT.xlsx')
df_clement = pd.read_excel('/Users/luke.thompson/Google Drive/My Drive/NOAA/okex/metadata/Okeanos_Explorer_Sample_Track_Clement_LRT.xlsx')
df_md5 = pd.read_csv('/Users/luke.thompson/Google Drive/My Drive/NOAA/okex/metadata/MD5/okex_md5.txt', sep='\s+', header=None, names=['checksum', 'filename'])

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/var/folders/rk/h_1wslx16sxbwsx5k1s2jn2h0000gn/T/ipykernel_11191/3786028939.py:4: SyntaxWarning: invalid escape sequence '\s'
  df_md5 = pd.read_csv('/Users/luke.thompson/Google Drive/My Drive/NOAA/okex/metadata/MD5/okex_md5.txt', sep='\s+', header=None, names=['checksum', 'filename'])


In [3]:
faire_columns_sample = ['samp_name', 'samp_category', 'neg_cont_type', 'pos_cont_type', 'materialSampleID', 'sample_derived_from', 'sample_composed_of', 'rel_cont_id', 'biological_rep_relation', 
 'decimalLongitude', 'decimalLatitude', 'verbatimLongitude', 'verbatimLatitude', 'verbatimCoordinateSystem', 'verbatimSRS', 'expedition_id', 'cruiseName', 'cruiseURL', 
 'cruiseStartDate', 'cruiseEndDate', 'cruiseLocation', 'geomeName', 'geomeURL', 'expeditionReportName', 'expeditionReportURL', 'geo_loc_name', 'eventDate', 'eventDurationValue', 
 'verbatimEventDate', 'verbatimEventTime', 'env_broad_scale', 'env_local_scale', 'env_medium', 'habitat_natural_artificial_0_1', 'samp_collect_method', 'samp_collect_device', 
 'samp_size', 'samp_size_unit', 'samp_store_temp', 'samp_store_sol', 'samp_store_dur', 'samp_store_loc', 'dna_store_loc', 'samp_store_method_additional', 'samp_mat_process', 
 'filter_passive_active_0_1', 'stationed_sample_dur', 'pump_flow_rate', 'pump_flow_rate_unit', 'prefilter_material', 'size_frac_low', 'size_frac', 'filter_diameter', 
 'filter_surface_area', 'filter_material', 'filter_name', 'precip_chem_prep', 'precip_force_prep', 'precip_time_prep', 'precip_temp_prep', 'prepped_samp_store_temp', 
 'prepped_samp_store_sol', 'prepped_samp_store_dur', 'prep_method_additional', 'date_ext', 'samp_vol_we_dna_ext', 'samp_vol_we_dna_ext_unit', 'nucl_acid_ext_lysis', 
 'nucl_acid_ext_sep', 'nucl_acid_ext', 'nucl_acid_ext_kit', 'nucl_acid_ext_modify', 'dna_cleanup_0_1', 'dna_cleanup_method', 'concentration', 'concentration_unit', 
 'concentration_method', 'ratioOfAbsorbance260_280', 'pool_dna_num', 'nucl_acid_ext_method_additional', 'samp_weather', 'minimumDepthInMeters', 'maximumDepthInMeters', 
 'tot_depth_water_col', 'elev', 'temp', 'chlorophyll', 'light_intensity', 'ph', 'ph_meth', 'salinity', 'suspend_part_matter', 'tidal_stage', 'turbidity', 'water_current', 
 'solar_irradiance', 'wind_direction', 'wind_speed', 'diss_inorg_carb', 'diss_inorg_carb_unit', 'diss_inorg_nitro', 'diss_inorg_nitro_unit', 'diss_org_carb', 
 'diss_org_carb_unit', 'diss_org_nitro', 'diss_org_nitro_unit', 'diss_oxygen', 'diss_oxygen_unit', 'tot_diss_nitro', 'tot_diss_nitro_unit', 'tot_inorg_nitro', 
 'tot_inorg_nitro_unit', 'tot_nitro', 'tot_nitro_unit', 'tot_part_carb', 'tot_part_carb_unit', 'tot_org_carb', 'tot_org_carb_unit', 'tot_org_c_meth', 'tot_nitro_content', 
 'tot_nitro_content_unit', 'tot_nitro_cont_meth', 'tot_carb', 'tot_carb_unit', 'part_org_carb', 'part_org_carb_unit', 'part_org_nitro', 'part_org_nitro_unit', 'nitrate', 
 'nitrate_unit', 'nitrite', 'nitrite_unit', 'nitro', 'nitro_unit', 'org_carb', 'org_carb_unit', 'org_matter', 'org_matter_unit', 'org_nitro', 'org_nitro_unit', 'dna_conc', 
 'dna_conc_unit', 'ammonium', 'ammonium_unit', 'carbonate', 'carbonate_unit', 'hydrogen_ion', 'hydrogen_ion_unit', 'nitrate_plus_nitrite', 'nitrate_plus_nitrite_unit', 
 'omega_arag', 'pco2', 'pco2_unit', 'phosphate', 'phosphate_unit', 'pressure', 'pressure_unit', 'silicate', 'silicate_unit', 'tot_alkalinity', 'tot_alkalinity_unit', 
 'transmittance', 'transmittance_unit', 'serial_number', 'line_id', 'station_id', 'ctd_cast_number', 'ctd_bottle_number', 'replicate_number', 'extract_id', 'extract_plate', 
 'extract_well_number', 'extract_well_position', 'biosample_accession', 'organism', 'samp_collect_notes', 'dna_yield', 'dna_yield_unit']

additional_columns_sample = ['bcid', 'collectorList', 'continentOcean', 'microHabitat']

faire_columns_experimentrun = ['assay_name', 'pcr_plate_id', 'lib_id', 'seq_run_id', 'lib_conc', 'lib_conc_unit', 'lib_conc_meth', 'phix_perc', 'mid_foreward', 'mid_reverse', 'filename', 'filename2', 
 'checksum_filename', 'checksum_filename2', 'associatedSequences', 'input_read_count']

### Preview the dataframes and example output

In [4]:
df_gdrive

,Index,expedition_id,cruiseName,cruiseURL,cruiseStartDate,cruiseEndDate,cruiseLocation,geomeName,geomeURL,rawReadsFolder,...,temp,diss_oxygen,diss_oxygen_unit,size_frac,samp_vol_we_dna_ext,samp_vol_we_dna_ext_unit,Preservation Method,Storage Method,Notes,Links
0,1,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,18.15,4.743,mg/L,0.45,1.7,L,missing: not provided,missing: not provided,NaN,NaN
1,2,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,4.722,5.252,mg/L,0.45,1.7,L,missing: not provided,missing: not provided,NaN,NaN
2,3,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,6.854,4.752,mg/L,0.45,1.7,L,missing: not provided,missing: not provided,NaN,NaN
3,4,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,7.67,4.743,mg/L,0.45,1.7,L,missing: not provided,missing: not provided,NaN,NaN
4,5,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,8.846,4.385,mg/L,0.45,1.7,L,missing: not provided,missing: not provided,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,462,EX2403,Beyond the Blue: Papahānaumokuākea Mapping 1,https://oceanexplorer.noaa.gov/okeanos/explora...,2024-07-21,2024-11-12,Pacific Ocean: Papahānaumokuākea Marine Nation...,EX2403_eDNA (EX2403),https://geome-db.org/record/ark:/21547/Fsv2,NaN,...,missing: not provided,missing: not provided,missing: not provided,0.45,1,L,DNA/RNA Shield,Refrigerated,homogenized sample for storage comparison expe...,EX2403 CTD001 Rosette Summary Form
462,463,EX2403,Beyond the Blue: Papahānaumokuākea Mapping 1,https://oceanexplorer.noaa.gov/okeanos/explora...,2024-07-21,2024-11-12,Pacific Ocean: Papahānaumokuākea Marine Nation...,EX2403_eDNA (EX2403),https://geome-db.org/record/ark:/21547/Fsv2,NaN,...,missing: not provided,missing: not provided,missing: not provided,0.45,1,L,DNA/RNA Shield,Room Temp,homogenized sample for storage comparison expe...,EX2403 CTD001 Rosette Summary Form
463,464,EX2403,Beyond the Blue: Papahānaumokuākea Mapping 1,https://oceanexplorer.noaa.gov/okeanos/explora...,2024-07-21,2024-11-12,Pacific Ocean: Papahānaumokuākea Marine Nation...,EX2403_eDNA (EX2403),https://geome-db.org/record/ark:/21547/Fsv2,NaN,...,missing: not provided,missing: not provided,missing: not provided,0.45,1,L,DNA/RNA Shield,Room Temp,homogenized sample for storage comparison expe...,EX2403 CTD001 Rosette Summary Form
464,465,EX2403,Beyond the Blue: Papahānaumokuākea Mapping 1,https://oceanexplorer.noaa.gov/okeanos/explora...,2024-07-21,2024-11-12,Pacific Ocean: Papahānaumokuākea Marine Nation...,EX2403_eDNA (EX2403),https://geome-db.org/record/ark:/21547/Fsv2,NaN,...,missing: not provided,missing: not provided,missing: not provided,0.45,1,L,DNA/RNA Shield,Room Temp,homogenized sample for storage comparison expe...,EX2403 CTD001 Rosette Summary Form


In [5]:
df_geome

,eventID,ShortSpec_ID,country,decimalLatitude,decimalLongitude,eventEnteredBy,locality,yearCollected,samplingProtocol,collectorList,...,environmentalMedium,permitType,permitText,permitURI,permitStatus,recordedByID,parentEventID,bcid,expeditionCode,projectId
0,EX2107_D01_01,EX2107_D01_01W,USA,31.21016,-77.85317,Allen Collins,Blake Plateau: Reef Tracts,2021,1.7 liters water by Niskin on D2,NOAA Ocean Exploration,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_01,EX2107_eDNA,575
1,EX2107_D01_02,EX2107_D01_02W,USA,31.21166,-77.85254,Allen Collins,Blake Plateau: Reef Tracts,2021,1.7 liters water by Niskin on D2,NOAA Ocean Exploration,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_02,EX2107_eDNA,575
2,EX2107_D01_06,EX2107_D01_06W,USA,31.21058,-77.85246,Allen Collins,Blake Plateau: Reef Tracts,2021,1.7 liters water by Niskin on D2,NOAA Ocean Exploration,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_06,EX2107_eDNA,575
3,EX2107_D01_11,EX2107_D01_11W,USA,31.21004,-77.85298,Allen Collins,Blake Plateau: Reef Tracts,2021,1.7 liters water by Niskin on D2,NOAA Ocean Exploration,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_11,EX2107_eDNA,575
4,EX2107_D01_13,EX2107_D01_13W,USA,31.21083,-77.85487,Allen Collins,Blake Plateau: Reef Tracts,2021,1.7 liters water by Niskin on D2,NOAA Ocean Exploration,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_13,EX2107_eDNA,575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,EX2403_CTD001_H20,EX2403_CTD001_H20W,USA,27.31582,-167.47420,Steve Auscavitch,North Pacific Ocean: PapahƒÅnaumokuƒÅkea Marin...,2024,1 liters of seawater filtered from Niskin on C...,NOAA Ocean Exploration,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fsx2EX2403_CTD001_H20,EX2403,575
459,EX2403_CTD001_H21,EX2403_CTD001_H21W,USA,27.31582,-167.47420,Steve Auscavitch,North Pacific Ocean: PapahƒÅnaumokuƒÅkea Marin...,2024,1 liters of seawater filtered from Niskin on C...,NOAA Ocean Exploration,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fsx2EX2403_CTD001_H21,EX2403,575
460,EX2403_CTD001_H22,EX2403_CTD001_H22W,USA,27.31582,-167.47420,Steve Auscavitch,North Pacific Ocean: PapahƒÅnaumokuƒÅkea Marin...,2024,1 liters of seawater filtered from Niskin on C...,NOAA Ocean Exploration,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fsx2EX2403_CTD001_H22,EX2403,575
461,EX2403_CTD001_H23,EX2403_CTD001_H23W,USA,27.31582,-167.47420,Steve Auscavitch,North Pacific Ocean: PapahƒÅnaumokuƒÅkea Marin...,2024,1 liters of seawater filtered from Niskin on C...,NOAA Ocean Exploration,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fsx2EX2403_CTD001_H23,EX2403,575


In [6]:
df_clement

,Data_type,Cruise,sample-id,filename (forward),lib_id (Tourmaline sample_name),lib_id_short (may be incorrect),ShortSpec_ID (may be incorrect),ShortSpec_ID (corrected),samp_name (corrected)
0,12S,OKEX2301_ROV-D2,OKEX2301_ROV-D10_BLK-1-1-12S_S53_L001,BLK-1-1-12S_S53_L001_R1_001.fastq.gz,BLK-1-1-12S_S53_L001,BLK-1-1-12S,not applicable: control sample,not applicable: control sample,EX2301_PCRBlank_1_12S
1,12S,OKEX2301_ROV-D2,OKEX2301_ROV-D100_D08-09-12S_S42_L001,D08-09-12S_S42_L001_R1_001.fastq.gz,D08-09-12S_S42_L001,D08-09-12S,EX2301_D08_09W,EX2301_D08_09W,EX2301_D08_09W
2,12S,OKEX2301_ROV-D2,OKEX2301_ROV-D102_D08-10-12S_S43_L001,D08-10-12S_S43_L001_R1_001.fastq.gz,D08-10-12S_S43_L001,D08-10-12S,EX2301_D08_10W,EX2301_D08_10W,EX2301_D08_10W
3,12S,OKEX2301_ROV-D2,OKEX2301_ROV-D104_D08-BLK-12S_S44_L001,D08-BLK-12S_S44_L001_R1_001.fastq.gz,D08-BLK-12S_S44_L001,D08-BLK-12S,EX2301_D08_BLW,EX2301_D08_BLW,EX2301_D08_BLW
4,12S,OKEX2301_ROV-D2,OKEX2301_ROV-D106_D09-04-12S_S45_L001,D09-04-12S_S45_L001_R1_001.fastq.gz,D09-04-12S_S45_L001,D09-04-12S,EX2301_D09_04W,EX2301_D09_04W,EX2301_D09_04W
...,...,...,...,...,...,...,...,...,...
703,COI,OKEX2301_ROV-D2,D09-05_S46_L001,D09-05_S46_L001_R1_001.fastq.gz,D09-05_S46_L001,D09-05,EX2301_D09_05W,EX2301_D09_05W,EX2301_D09_05W
704,COI,OKEX2301_ROV-D2,D09-06_S47_L001,D09-06_S47_L001_R1_001.fastq.gz,D09-06_S47_L001,D09-06,EX2301_D09_06W,EX2301_D09_06W,EX2301_D09_06W
705,COI,OKEX2301_ROV-D2,D09-07_S48_L001,D09-07_S48_L001_R1_001.fastq.gz,D09-07_S48_L001,D09-07,EX2301_D09_07W,EX2301_D09_07W,EX2301_D09_07W
706,COI,OKEX2301_ROV-D2,D09-08_S49_L001,D09-08_S49_L001_R1_001.fastq.gz,D09-08_S49_L001,D09-08,EX2301_D09_08W,EX2301_D09_08W,EX2301_D09_08W


In [7]:
df_md5

,checksum,filename
0,940fc5832d2fb9c076f6755bd80033cb,EX2201_D01_01_18S_S288_R1_001.fastq.gz
1,2c64b8020053fc3c34767f6727e2fd9b,EX2201_D01_01_18S_S288_R2_001.fastq.gz
2,404f8422bf60ec6f1fc4add09fd5b0fe,EX2201_D01_Blank_18S_S289_R1_001.fastq.gz
3,bd06fbde00a62bc70991065cdda38bc6,EX2201_D01_Blank_18S_S289_R2_001.fastq.gz
4,f3ab159267bba2a076b1da3d580836e0,EX2201_D04_01_18S_S290_R1_001.fastq.gz
...,...,...
1538,315c3e2f18ca453890bc7894e38ec8da,D09-07_S48_L001_R2_001.fastq.gz
1539,a0d85294c895e9d40f38b5143c9daf55,D09-08_S49_L001_R1_001.fastq.gz
1540,0dcc896ed64e093c6db9e2a5ac2704e4,D09-08_S49_L001_R2_001.fastq.gz
1541,eb61d35c74cf20d0877103821bd58121,D09-BLK_S50_L001_R1_001.fastq.gz


### Merge the dataframes

In [8]:
# merge df_gdrive with df_geome
df_metadata = pd.merge(df_gdrive, df_geome, how='outer', on='ShortSpec_ID', suffixes=('_gdrive', '_geome'))

In [9]:
df_metadata

,Index,expedition_id,cruiseName,cruiseURL,cruiseStartDate,cruiseEndDate,cruiseLocation,geomeName,geomeURL,rawReadsFolder,...,environmentalMedium,permitType,permitText,permitURI,permitStatus,recordedByID,parentEventID,bcid,expeditionCode,projectId
0,1.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_01,EX2107_eDNA,575.0
1,2.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_02,EX2107_eDNA,575.0
2,3.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_06,EX2107_eDNA,575.0
3,4.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_11,EX2107_eDNA,575.0
4,5.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,sea water,NaN,NaN,NaN,NaN,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_13,EX2107_eDNA,575.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,182.0,EX2205,Voyage to the Ridge 2022 [Expedition 2],https://oceanexplorer.noaa.gov/okeanos/explora...,2022-06-16,2022-07-13,Central North Atlantic,EX2205_eDNA (EX2205_eDNA),https://geome-db.org/record/ark:/21547/Fil2,EX2205part,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,183.0,EX2205,Voyage to the Ridge 2022 [Expedition 2],https://oceanexplorer.noaa.gov/okeanos/explora...,2022-06-16,2022-07-13,Central North Atlantic,EX2205_eDNA (EX2205_eDNA),https://geome-db.org/record/ark:/21547/Fil2,EX2205part,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,184.0,EX2205,Voyage to the Ridge 2022 [Expedition 2],https://oceanexplorer.noaa.gov/okeanos/explora...,2022-06-16,2022-07-13,Central North Atlantic,EX2205_eDNA (EX2205_eDNA),https://geome-db.org/record/ark:/21547/Fil2,EX2205part,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,185.0,EX2205,Voyage to the Ridge 2022 [Expedition 2],https://oceanexplorer.noaa.gov/okeanos/explora...,2022-06-16,2022-07-13,Central North Atlantic,EX2205_eDNA (EX2205_eDNA),https://geome-db.org/record/ark:/21547/Fil2,EX2205part,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# merge df_metadata with df_clement
df_metadata2 = pd.merge(df_metadata, df_clement, how='outer', left_on='ShortSpec_ID', right_on='ShortSpec_ID (corrected)', suffixes=('_gdrive', '_clement'))
# drop rows where 'samp_name (corrected)' is NaN
df_metadata2 = df_metadata2.dropna(subset=['samp_name (corrected)'])

### FUNCTIONS - GENERAL

# function to add new column with arbitrary value in each row
def add_missing_column(row, value):
    return value

# function to get sample category based on 'samp_name (corrected)'
def get_samp_category(name):
    name_str = str(name)
    if 'Blank' in name_str or 'BLW' in name_str:
        return 'negative control'
    else:
        return 'sample'

### FUNCTIONS - BASED ON SAMP_NAME (CORRECTED) OR SAMP_CATEGORY

# function to get negative control type based on 'samp_name (corrected)'
def get_neg_control_type(name):
    name_str = str(name)
    if 'PCRBlank' in name_str or 'IndexBlank' in name_str:
        return 'PCR negative'
    elif 'FieldBlank' in name_str or 'BLW' in name_str:
        return 'field negative'
    elif 'ExtractBlank' in name_str:
        return 'extraction negative'
    else:
        return None

# function to get decimal latitude from either gdrive or geome
def get_decimal_latitude(row):
    lat_gdrive = pd.to_numeric(row['decimalLatitude_gdrive'], errors='coerce')
    lat_geome = pd.to_numeric(row['decimalLatitude_geome'], errors='coerce')
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        return 'not applicable: control sample'
    elif not pd.isna(lat_gdrive):
        return lat_gdrive
    elif not pd.isna(lat_geome):
        return lat_geome
    else:
        return 'not applicable: control sample'

# function to get decimal longitude from either gdrive or geome
def get_decimal_longitude(row):
    lon_gdrive = pd.to_numeric(row['decimalLongitude_gdrive'], errors='coerce')
    lon_geome = pd.to_numeric(row['decimalLongitude_geome'], errors='coerce')
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        return 'not applicable: control sample'
    elif not pd.isna(lon_gdrive):
        return lon_gdrive
    elif not pd.isna(lon_geome):
        return lon_geome
    else:
        return 'not applicable: control sample'
   
# function to get minimumDepthInMeters from either gdrive or geome
def get_minimum_depth(row):
    depth_gdrive = pd.to_numeric(row['minimumDepthInMeters_gdrive'], errors='coerce')
    depth_geome = pd.to_numeric(row['minimumDepthInMeters_geome'], errors='coerce')
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        return 'not applicable: control sample'
    elif not pd.isna(depth_gdrive):
        return depth_gdrive
    elif not pd.isna(depth_geome):
        return depth_geome
    else:
        return 'missing: not provided'

# function to get maximumDepthInMeters from either gdrive or geome
def get_maximum_depth(row):
    depth_gdrive = pd.to_numeric(row['maximumDepthInMeters_gdrive'], errors='coerce')
    depth_geome = pd.to_numeric(row['maximumDepthInMeters_geome'], errors='coerce')
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        return 'not applicable: control sample'
    elif not pd.isna(depth_gdrive):
        return depth_gdrive
    elif not pd.isna(depth_geome):
        return depth_geome
    else:
        return 'missing: not provided'

# function to get env_broad_scale based on samp_category
def get_env_broad_scale(row):
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        return 'not applicable: negative control'
    elif samp_category == 'sample':
        return 'marine biome [ENVO:00000447]'
    else:
        return 'missing: not provided'

# function to get env_local_scale based on samp_category and minimumDepthInMeters ([0,200) is epipelagic, [200,1000) is mesopelagic, [1000,) is bathypelagic)
def get_env_local_scale(row):
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        return 'not applicable: negative control'
    elif samp_category == 'sample':
        if row['minimumDepthInMeters'] < 200:
            return 'marine epipelagic zone [ENVO:00000212]'
        elif row['minimumDepthInMeters'] < 1000:
            return 'marine mesopelagic zone [ENVO:00000213]'
        elif row['minimumDepthInMeters'] >= 1000:
            return 'marine bathypelagic zone [ENVO:00000214]'
    else:
        return 'missing: not provided'

# function to get env_medium based on samp_category
def get_env_medium(row):
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        return 'not applicable: negative control'
    elif samp_category == 'sample':
        return 'sea water [ENVO:00002149]'
    else:
        return 'missing: not provided'

# function to get habitat_natural_artificial_0_1 based on samp_category
def get_habitat_natural_artificial_0_1(row):
    samp_category = row['samp_category']
    if samp_category == 'sample':
        return '0'
    elif samp_category == 'negative control':
        return 'not applicable: control sample'
    else:
        return 'missing: not provided'

### FUNCTIONS - OTHER

# function to get expedition ID from 'expedition_id' if it exists, otherwise extract EX[0-9]{4} pattern from 'expeditionCode' or 'Cruise'
def get_expedition_id(row):
    if pd.notna(row['expedition_id']):
        return row['expedition_id']
    elif pd.notna(row['expeditionCode']) and re.search(r'EX\d{4}', str(row['expeditionCode'])):
        match = re.search(r'EX\d{4}', str(row['expeditionCode']))
        return match.group()
    elif pd.notna(row['Cruise']) and re.search(r'EX\d{4}', str(row['Cruise'])):
        match = re.search(r'EX\d{4}', str(row['Cruise']))
        return match.group()
    else:
        return 'missing: not provided'

# function to fill in missing cruise information: for any rows missing 'cruiseName', 'cruiseURL', 'cruiseStartDate', 'cruiseEndDate', 
# 'cruiseLocation', 'geomeName', 'geomeURL', 'expeditionReportName', or 'expeditionReportURL',
# fill in with values from other rows with the same 'expedition_id'
def fill_missing_cruise_info(row, df):
    if pd.isna(row['cruiseName']) or pd.isna(row['cruiseURL']) or pd.isna(row['cruiseStartDate']) or pd.isna(row['cruiseEndDate']) or pd.isna(row['cruiseLocation']) or pd.isna(row['geomeName']) or pd.isna(row['geomeURL']) or pd.isna(row['expeditionReportName']) or pd.isna(row['expeditionReportURL']):
        expedition_id = row['expedition_id']
        matching_rows = df[df['expedition_id'] == expedition_id]
        if not matching_rows.empty:
            row['cruiseName'] = matching_rows['cruiseName'].iloc[0] if pd.isna(row['cruiseName']) else row['cruiseName']
            row['cruiseURL'] = matching_rows['cruiseURL'].iloc[0] if pd.isna(row['cruiseURL']) else row['cruiseURL']
            row['cruiseStartDate'] = matching_rows['cruiseStartDate'].iloc[0] if pd.isna(row['cruiseStartDate']) else row['cruiseStartDate']
            row['cruiseEndDate'] = matching_rows['cruiseEndDate'].iloc[0] if pd.isna(row['cruiseEndDate']) else row['cruiseEndDate']
            row['cruiseLocation'] = matching_rows['cruiseLocation'].iloc[0] if pd.isna(row['cruiseLocation']) else row['cruiseLocation']
            row['geomeName'] = matching_rows['geomeName'].iloc[0] if pd.isna(row['geomeName']) else row['geomeName']
            row['geomeURL'] = matching_rows['geomeURL'].iloc[0] if pd.isna(row['geomeURL']) else row['geomeURL']
            row['expeditionReportName'] = matching_rows['expeditionReportName'].iloc[0] if pd.isna(row['expeditionReportName']) else row['expeditionReportName']
            row['expeditionReportURL'] = matching_rows['expeditionReportURL'].iloc[0] if pd.isna(row['expeditionReportURL']) else row['expeditionReportURL']
    return row

# function to fill in missing 'geo_loc_name' with 'locality' if exists, else return 'not applicable: control sample' if samp_category is 'negative control' or 'missing: not provided' if samp_category is 'sample'
def fill_missing_geo_loc_name(row, df):
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        row['geo_loc_name'] = 'not applicable: control sample'
    elif pd.isna(row['geo_loc_name']):
        locality = row['locality'] if 'locality' in row else None
        if locality:
            row['geo_loc_name'] = locality
        else:
            row['geo_loc_name'] = 'missing: not provided'
    return row

# function to check if a string can be converted to datetime using pandas
def is_valid_datetime(date_string):
    """Check if string can be converted to datetime using pandas"""
    try:
        result = pd.to_datetime(date_string, errors='coerce')
        return pd.notna(result)
    except:
        return False

# function to get verbatim eventDate from column eventDate if valid datetime else timeOfDay else "missing: not provided"
def get_verbatim_event_datetime(row):
    samp_category = row['samp_category']
    if is_valid_datetime(row['eventDate']):
        return row['eventDate']
    elif 'timeOfDay' in row and pd.notna(row['timeOfDay']):
        return row['timeOfDay']
    elif 'yearCollected' in row and pd.notna(row['yearCollected']) and 'monthCollected' in row and pd.notna(row['monthCollected']) and 'dayCollected' in row and pd.notna(row['dayCollected']):
        year = int(row['yearCollected'])
        month = int(row['monthCollected'])
        day = int(row['dayCollected'])
        return f"{year}, {month:02d}, {day:02d}"
    elif samp_category == 'negative control':
        return 'not applicable: control sample'
    else:
        return 'missing: not provided'

# function to fill in missing or non-datetime 'eventDate' with values from 'yearCollected', 'monthCollected', and 'dayCollected', converting them to integer and then string format first, else return 'not applicable: control sample' if samp_category is 'negative control' or 'missing: not provided' if samp_category is 'sample'
def fill_missing_event_date(row):
    samp_category = row['samp_category']
    if not is_valid_datetime(row['eventDate']):
        if samp_category == 'negative control':
            row['eventDate'] = 'not applicable: control sample'
        else:
            # Attempt to construct eventDate from year, month, and day
            year = int(row['yearCollected']) if 'yearCollected' in row and pd.notna(row['yearCollected']) else None
            month = int(row['monthCollected']) if 'monthCollected' in row and pd.notna(row['monthCollected']) else None
            day = int(row['dayCollected']) if 'dayCollected' in row and pd.notna(row['dayCollected']) else None
            if year and month and day:
                row['eventDate'] = f"{year}-{month:02d}-{day:02d}"
            else:
                row['eventDate'] = 'missing: not provided'
    return row

# function to fill in missing 'samp_collect_device' with 'not applicable: control sample' if it is NaN or samp_category is 'negative control'
def fill_missing_samp_collect_device(row):
    if pd.isna(row['samp_collect_device']) or row['samp_category'] == 'negative control':
        return 'not applicable: control sample'
    return row['samp_collect_device']

# function to add column 'samp_collect_method' based on 'samp_collect_device'
def get_samp_collect_method(row):
    samp_collect_method = row['samplingProtocol']
    samp_collect_device = row['samp_collect_device']
    if pd.isna(samp_collect_device) or samp_collect_device == 'not applicable: control sample':
        return 'not applicable: control sample'
    elif pd.notna(samp_collect_method):
        return samp_collect_method
    elif 'CTD' in samp_collect_device:
        return 'CTD rosette system on Okeanos Explorer'
    elif 'ROV' in samp_collect_device:
        return '1.7 liters water by Niskin on D2'
    else:
        return 'missing: not provided'

# function to add column 'samp_size' based on 'samp_collect_device'
def get_samp_size(row):
    samp_collect_device = row['samp_collect_device']
    if pd.isna(samp_collect_device) or samp_collect_device == 'not applicable: control sample':
        return 'not applicable: control sample'
    elif 'CTD' in samp_collect_device:
        return '10'
    elif 'ROV' in samp_collect_device:
        return '1.7'
    else:
        return 'missing: not provided'

# function to add column 'samp_size_unit' based on 'samp_collect_device'
def get_samp_size_unit(row):
    samp_collect_device = row['samp_collect_device']
    if pd.isna(samp_collect_device) or samp_collect_device == 'not applicable: control sample':
        return 'not applicable: control sample'
    elif 'CTD' in samp_collect_device:
        return 'L'
    elif 'ROV' in samp_collect_device:
        return 'L'
    else:
        return 'missing: not provided'

# function to add column 'samp_collect_notes' that is concatenation of 'fieldIdentification' and 'Notes' and 'eventRemarks' if exists
def get_samp_collect_notes(row):
    field_id = row['fieldIdentification'] if 'fieldIdentification' in row and pd.notna(row['fieldIdentification']) else ''
    notes = row['Notes'] if 'Notes' in row and pd.notna(row['Notes']) else ''
    event_remarks = row['eventRemarks'] if 'eventRemarks' in row and pd.notna(row['eventRemarks']) else ''
    return f"{field_id} {notes} {event_remarks}".strip()
   
# function to add column 'ctd_cast_number' based on 'DiveData_ID' if contains 'DIVE', 'Rosette_No' if contains 'CTD' or 'ROS', otherwise 'not applicable: sample group'
def get_ctd_cast_number(row):
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        return 'not applicable: control sample'
    elif 'DiveData_ID' in row and pd.notna(row['DiveData_ID']) and 'DIVE' in row['DiveData_ID']:
        return row['DiveData_ID']
    elif 'Rosette_No' in row and pd.notna(row['Rosette_No']) and ('CTD' in row['Rosette_No'] or 'ROS' in row['Rosette_No']):
        return row['Rosette_No']
    else:
        return 'not applicable: sample group'

# function to add column 'ctd_bottle_number' based on 'ShortSpec_ID' portion that contains 'B[0-9]{2}', otherwise 'not applicable: sample group'
def get_ctd_bottle_number(row):
    samp_category = row['samp_category']
    if samp_category == 'negative control':
        return 'not applicable: control sample'
    elif 'ShortSpec_ID' in row and pd.notna(row['ShortSpec_ID']):
        match = re.search(r'B[0-9]{2}', row['ShortSpec_ID'])
        if match:
            return match.group(0)
        else:
            return 'not applicable: sample group'
    else:
        return 'not applicable: sample group'
   
# function to add column 'assay_name' based on 'Data_type' if it exists, otherwise 'missing: not provided', with mappings: 16S:Bacteria-16S-V4-EMP, 18S:Eukarya-18S-V4-Stoeck-Piredda, 18S(expedition_id=EX2301):Eukarya-18S-V9-AmaralZettler, 12S:Fish-12S-MiFish-Miya, COI:Metazoa-COI-Leray-Geller, 28S:Anthozoa-28S-McCartin
def get_assay_name(row):
    if 'Data_type' in row and pd.notna(row['Data_type']):
        data_type = row['Data_type']
        expedition_id = row['expedition_id'] if 'expedition_id' in row else None
        if data_type == '16S':
            return 'Bacteria-16S-V4-EMP'
        elif data_type == '18S':
            if expedition_id == 'EX2301':
                return 'Eukarya-18S-V9-AmaralZettler'
            else:
                return 'Eukarya-18S-V4-Stoeck-Piredda'
        elif data_type == '12S':
            return 'Fish-12S-MiFish-Miya'
        elif data_type == 'COI':
            return 'Metazoa-COI-Leray-Geller'
        elif data_type == '28S':
            return 'Anthozoa-28S-McCartin'
    return 'missing: not provided'

# function to add column 'filename2' based on 'filename (forward)' if it exists, replacing '_R1.fastq.gz' with '_R2.fastq.gz' and '_R1_001.fastq.gz' with '_R2_001.fastq.gz', otherwise 'missing: not provided'
def get_filename2(row):
    if 'filename (forward)' in row and pd.notna(row['filename (forward)']):
        return row['filename (forward)'].replace('_R1.fastq.gz', '_R2.fastq.gz').replace('_R1_001.fastq.gz', '_R2_001.fastq.gz')
    else:
        return 'missing: not provided'

# function to add md5 checksum from df_md5 based on filenames in column x
def get_md5_checksum(row, column):
    filename = row[column] if column in row and pd.notna(row[column]) else None
    if filename:
        md5_row = df_md5[df_md5['filename'] == filename]
        if not md5_row.empty:
            return md5_row['checksum'].values[0]
    return 'missing: not provided'

### run the functions to correct or fill in missing data

# add column 'samp_name' using 'samp_name (corrected)'
df_metadata2['samp_name'] = df_metadata2['samp_name (corrected)']

# add columns 'materialSampleID' and 'serial_number' using 'ShortSpec_ID (corrected)'
df_metadata2['materialSampleID'] = df_metadata2['ShortSpec_ID (corrected)']
df_metadata2['serial_number'] = df_metadata2['ShortSpec_ID (corrected)']

# add column 'samp_category' using function get_samp_category
df_metadata2['samp_category'] = df_metadata2['samp_name (corrected)'].apply(get_samp_category)

# add column 'neg_cont_type' using function get_neg_control_type
df_metadata2['neg_cont_type'] = df_metadata2['samp_name (corrected)'].apply(get_neg_control_type)

# add column 'decimalLatitude' using function get_decimal_latitude
df_metadata2['decimalLatitude'] = df_metadata2.apply(get_decimal_latitude, axis=1)

# add column 'decimalLongitude' using function get_decimal_longitude
df_metadata2['decimalLongitude'] = df_metadata2.apply(get_decimal_longitude, axis=1)

# add column 'minimumDepthInMeters' using function get_minimum_depth
df_metadata2['minimumDepthInMeters'] = df_metadata2.apply(get_minimum_depth, axis=1)

# add column 'maximumDepthInMeters' using function get_maximum_depth
df_metadata2['maximumDepthInMeters'] = df_metadata2.apply(get_maximum_depth, axis=1)

# add column 'tot_depth_water_col' using 'depthOfBottomInMeters'
df_metadata2['tot_depth_water_col'] = df_metadata2['depthOfBottomInMeters'].fillna('missing: not provided')

# fix column 'expedition_id' using function get_expedition_id
df_metadata2['expedition_id'] = df_metadata2.apply(get_expedition_id, axis=1)

# fill missing cruise information
df_metadata2 = df_metadata2.apply(lambda row: fill_missing_cruise_info(row, df_metadata2), axis=1)

# fill missing 'geo_loc_name' with 'locality' if exists
df_metadata2 = df_metadata2.apply(lambda row: fill_missing_geo_loc_name(row, df_metadata2), axis=1)

# add column 'eventDurationValue' with 'missing: not collected' in each row
df_metadata2['eventDurationValue'] = df_metadata2.apply(lambda row: add_missing_column(row, 'missing: not collected'), axis=1)

# add column 'verbatimEventDate' using function get_verbatim_event_datetime
df_metadata2['verbatimEventDate'] = df_metadata2.apply(get_verbatim_event_datetime, axis=1)

# add column 'verbatimEventTime' using function get_verbatim_event_datetime
df_metadata2['verbatimEventTime'] = df_metadata2.apply(get_verbatim_event_datetime, axis=1)
# change any values of 'verbatimEventTime' containing a comma (special case) to 'missing: not provided'
df_metadata2['verbatimEventTime'] = df_metadata2['verbatimEventTime'].apply(lambda x: 'missing: not provided' if ',' in str(x) else x)

# fill missing 'eventDate' with values from 'yearCollected', 'monthCollected', 'dayCollected'
df_metadata2 = df_metadata2.apply(fill_missing_event_date, axis=1)

# add column env_broad_scale using function get_env_broad_scale
df_metadata2['env_broad_scale'] = df_metadata2.apply(get_env_broad_scale, axis=1)

# add column env_local_scale using function get_env_local_scale
df_metadata2['env_local_scale'] = df_metadata2.apply(get_env_local_scale, axis=1)

# add column env_medium using function get_env_medium
df_metadata2['env_medium'] = df_metadata2.apply(get_env_medium, axis=1)

# add column habitat_natural_artificial_0_1 using function get_habitat_natural_artificial_0_1
df_metadata2['habitat_natural_artificial_0_1'] = df_metadata2.apply(get_habitat_natural_artificial_0_1, axis=1)

# fill missing 'samp_collect_device' with 'not applicable: control sample' if it is NaN
df_metadata2['samp_collect_device'] = df_metadata2.apply(fill_missing_samp_collect_device, axis=1)

# add column 'samp_collect_method' using function get_samp_collect_method
df_metadata2['samp_collect_method'] = df_metadata2.apply(get_samp_collect_method, axis=1)

# add column 'samp_size' using function get_samp_size
df_metadata2['samp_size'] = df_metadata2.apply(get_samp_size, axis=1)

# add column 'samp_size_unit' using function get_samp_size_unit
df_metadata2['samp_size_unit'] = df_metadata2.apply(get_samp_size_unit, axis=1)

# add column 'samp_collect_notes' using function get_samp_collect_notes
df_metadata2['samp_collect_notes'] = df_metadata2.apply(get_samp_collect_notes, axis=1)

# add column 'ctd_cast_number' using function get_ctd_cast_number
df_metadata2['ctd_cast_number'] = df_metadata2.apply(get_ctd_cast_number, axis=1)

# add column 'ctd_bottle_number' using function get_ctd_bottle_number
df_metadata2['ctd_bottle_number'] = df_metadata2.apply(get_ctd_bottle_number, axis=1)

# add column 'assay_name' using function get_assay_name
df_metadata2['assay_name'] = df_metadata2.apply(get_assay_name, axis=1)

# add column 'seq_run_id' from 'Cruise' if it exists, otherwise 'missing: not provided'
df_metadata2['seq_run_id'] = df_metadata2['Cruise'].fillna('missing: not provided')

# add column 'lib_id' from 'lib_id (Tourmaline sample_name)' if it exists, otherwise 'missing: not provided'
df_metadata2['lib_id'] = df_metadata2['lib_id (Tourmaline sample_name)'].fillna('missing: not provided')

# add column 'filename' from 'filename (forward)' if it exists, otherwise 'missing: not provided'
df_metadata2['filename'] = df_metadata2['filename (forward)'].fillna('missing: not provided')

# add column 'filename2' using function get_filename2
df_metadata2['filename2'] = df_metadata2.apply(get_filename2, axis=1)

# add column 'checksum_filename' using function get_md5_checksum for 'filename'
df_metadata2['checksum_filename'] = df_metadata2.apply(lambda row: get_md5_checksum(row, 'filename'), axis=1)

# add column 'checksum_filename2' using function get_md5_checksum for 'filename2'
df_metadata2['checksum_filename2'] = df_metadata2.apply(lambda row: get_md5_checksum(row, 'filename2'), axis=1)

In [11]:
df_metadata2

,Index,expedition_id,cruiseName,cruiseURL,cruiseStartDate,cruiseEndDate,cruiseLocation,geomeName,geomeURL,rawReadsFolder,...,samp_collect_notes,ctd_cast_number,ctd_bottle_number,assay_name,seq_run_id,lib_id,filename,filename2,checksum_filename,checksum_filename2
0,1.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,,DIVE01,not applicable: sample group,Eukarya-18S-V4-Stoeck-Piredda,OKEX2107part-OKEX2205part_ROV-D2,18S-EX2107D1-01_S1_L001,18S-EX2107D1-01_S1_L001_R1_001.fastq.gz,18S-EX2107D1-01_S1_L001_R2_001.fastq.gz,ed08d22bcf9d76327f2d60c85f93c454,ca881cff4725a51451c9689e6dbe14f2
1,1.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,,DIVE01,not applicable: sample group,Metazoa-COI-Leray-Geller,OKEX2107part-OKEX2205part_ROV-D2,COI-EX2107D1-01_S51_L001,COI-EX2107D1-01_S51_L001_R1_001.fastq.gz,COI-EX2107D1-01_S51_L001_R2_001.fastq.gz,a509ccfef32696dcd6495c56baf95261,fb7d290d0c2df76e578824d6a9e984e4
2,2.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,,DIVE01,not applicable: sample group,Eukarya-18S-V4-Stoeck-Piredda,OKEX2107part-OKEX2205part_ROV-D2,18S-EX2107D1-02_S2_L001,18S-EX2107D1-02_S2_L001_R1_001.fastq.gz,18S-EX2107D1-02_S2_L001_R2_001.fastq.gz,5d5f0fc74288f5048b3111709791dcd3,8cdaaa859f687616c1cdbc6f940064e8
3,2.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,,DIVE01,not applicable: sample group,Metazoa-COI-Leray-Geller,OKEX2107part-OKEX2205part_ROV-D2,COI-EX2107D1-02_S52_L001,COI-EX2107D1-02_S52_L001_R1_001.fastq.gz,COI-EX2107D1-02_S52_L001_R2_001.fastq.gz,180e22035e14011ddb3f4dd7c03742f9,1ce9393336d1a4bafa7ff599add758ea
4,3.0,EX2107,Windows to the Deep 2021: Southeast U.S. ROV a...,https://oceanexplorer.noaa.gov/okeanos/explora...,2021-10-26,2021-11-15,Southeast U.S.,EX2107_eDNA (EX2107_eDNA),https://geome-db.org/record/ark:/21547/Ffd2,OKEX2107part;OKEX2107part-OKEX2205part_ROV-D2,...,,DIVE01,not applicable: sample group,Eukarya-18S-V4-Stoeck-Piredda,OKEX2107part-OKEX2205part_ROV-D2,18S-EX2107D1-06_S3_L001,18S-EX2107D1-06_S3_L001_R1_001.fastq.gz,18S-EX2107D1-06_S3_L001_R2_001.fastq.gz,7aea11d84cd0988207fc9c550139f41b,b540dc64df1e353b2bf676e7b1900072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,NaN,EX2203,2022 Puerto Rico Mapping and Deep-Sea Camera D...,https://oceanexplorer.noaa.gov/okeanos/explora...,2022-04-02,2022-04-26,West Atlantic,EX2203_eDNA (EX2203_eDNA),https://geome-db.org/record/ark:/21547/Fid2,NaN,...,,not applicable: control sample,not applicable: control sample,Metazoa-COI-Leray-Geller,OKEX2203_CTD,EX2203-NegativeControl-COI_S48_L001,EX2203-NegativeControl-COI_S48_L001_R1_001.fas...,EX2203-NegativeControl-COI_S48_L001_R2_001.fas...,68eff05234e9196df0e05e8ddb219014,5870dba1b7d6f9434d74e017f8ae36c8
973,NaN,EX2301,2023 Shakedown + EXPRESS West Coast Exploration,https://oceanexplorer.noaa.gov/okeanos/explora...,2023-04-13,2023-04-27,U.S. Westcoast,EX2301_eDNA (EX2301_eDNA),https://geome-db.org/record/ark:/21547/FjB2,NaN,...,,not applicable: control sample,not applicable: control sample,Metazoa-COI-Leray-Geller,OKEX2301_ROV-D2,BLK-01_S25_L001,BLK-01_S25_L001_R1_001.fastq.gz,BLK-01_S25_L001_R2_001.fastq.gz,3a3580f73

### Reorder columns to match FAIRe-ODE Google Sheet

In [12]:
# create missing columns with NaN values if they don't exist
all_columns = faire_columns_sample + additional_columns_sample + faire_columns_experimentrun
for col in all_columns:
    if col not in df_metadata2.columns:
        df_metadata2[col] = np.nan

# reorder columns to match the order in all_columns
df_metadata3 = df_metadata2[all_columns]

In [13]:
df_metadata3

,samp_name,samp_category,neg_cont_type,pos_cont_type,materialSampleID,sample_derived_from,sample_composed_of,rel_cont_id,biological_rep_relation,decimalLongitude,...,lib_conc_meth,phix_perc,mid_foreward,mid_reverse,filename,filename2,checksum_filename,checksum_filename2,associatedSequences,input_read_count
0,EX2107_D01_01W,sample,None,NaN,EX2107_D01_01W,NaN,NaN,NaN,NaN,-77.8532,...,NaN,NaN,NaN,NaN,18S-EX2107D1-01_S1_L001_R1_001.fastq.gz,18S-EX2107D1-01_S1_L001_R2_001.fastq.gz,ed08d22bcf9d76327f2d60c85f93c454,ca881cff4725a51451c9689e6dbe14f2,NaN,NaN
1,EX2107_D01_01W,sample,None,NaN,EX2107_D01_01W,NaN,NaN,NaN,NaN,-77.8532,...,NaN,NaN,NaN,NaN,COI-EX2107D1-01_S51_L001_R1_001.fastq.gz,COI-EX2107D1-01_S51_L001_R2_001.fastq.gz,a509ccfef32696dcd6495c56baf95261,fb7d290d0c2df76e578824d6a9e984e4,NaN,NaN
2,EX2107_D01_02W,sample,None,NaN,EX2107_D01_02W,NaN,NaN,NaN,NaN,-77.8525,...,NaN,NaN,NaN,NaN,18S-EX2107D1-02_S2_L001_R1_001.fastq.gz,18S-EX2107D1-02_S2_L001_R2_001.fastq.gz,5d5f0fc74288f5048b3111709791dcd3,8cdaaa859f687616c1cdbc6f940064e8,NaN,NaN
3,EX2107_D01_02W,sample,None,NaN,EX2107_D01_02W,NaN,NaN,NaN,NaN,-77.8525,...,NaN,NaN,NaN,NaN,COI-EX2107D1-02_S52_L001_R1_001.fastq.gz,COI-EX2107D1-02_S52_L001_R2_001.fastq.gz,180e22035e14011ddb3f4dd7c03742f9,1ce9393336d1a4bafa7ff599add758ea,NaN,NaN
4,EX2107_D01_06W,sample,None,NaN,EX2107_D01_06W,NaN,NaN,NaN,NaN,-77.8525,...,NaN,NaN,NaN,NaN,18S-EX2107D1-06_S3_L001_R1_001.fastq.gz,18S-EX2107D1-06_S3_L001_R2_001.fastq.gz,7aea11d84cd0988207fc9c550139f41b,b540dc64df1e353b2bf676e7b1900072,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,EX2203_PCRBlank_COI,negative control,PCR negative,NaN,not applicable: control sample,NaN,NaN,NaN,NaN,not applicable: control sample,...,NaN,NaN,NaN,NaN,EX2203-NegativeControl-COI_S48_L001_R1_001.fas...,EX2203-NegativeControl-COI_S48_L001_R2_001.fas...,68eff05234e9196df0e05e8ddb219014,5870dba1b7d6f9434d74e017f8ae36c8,NaN,NaN
973,EX2301_PCRBlank_1_COI,negative control,PCR negative,NaN,not applicable: control sample,NaN,NaN,NaN,NaN,not applicable: control sample,...,NaN,NaN,NaN,NaN,BLK-01_S25_L001_R1_001.fastq.gz,BLK-01_S25_L001_R2_001.fastq.gz,3a3580f7373a411cd99d3f6d604d6d76,d57c5e701e4797a2d41e076053c45108,NaN,NaN
974,EX2301_PCRBlank_2_COI,negative control,PCR negative,NaN,not applicable: control sample,NaN,NaN,NaN,NaN,not applicable: control sample,...,NaN,NaN,NaN,NaN,BLK-02_S26_L001_R1_001.fastq.gz,BLK-02_S26_L001_R2_001.fastq.gz,3882b35ad415f3186dbda9f127ebc86d,3882b35ad415f3186dbda9f127ebc86d,NaN,NaN
975,EX2301_PCRBlank_3_COI,negative control,PCR negative,NaN,not applicable: control sample,NaN,NaN,NaN,NaN,not applicable: control sample,...,NaN,NaN,NaN,NaN,BLK-03_S51_L001_R1_001.fastq.gz,BLK-03_S51_L001_R2_001.fastq.gz,9a7d41e58e77cf96addeaa92f792dfca,61f799dd6e1ec4b94abbec3d130ea81f,NaN,NaN


### Split metadata into samples and libraries

In [14]:
df_samples = df_metadata3.drop(columns=faire_columns_experimentrun, errors='ignore').drop_duplicates()

In [15]:
df_samples

,samp_name,samp_category,neg_cont_type,pos_cont_type,materialSampleID,sample_derived_from,sample_composed_of,rel_cont_id,biological_rep_relation,decimalLongitude,...,extract_well_position,biosample_accession,organism,samp_collect_notes,dna_yield,dna_yield_unit,bcid,collectorList,continentOcean,microHabitat
0,EX2107_D01_01W,sample,None,NaN,EX2107_D01_01W,NaN,NaN,NaN,NaN,-77.8532,...,NaN,NaN,NaN,,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_01,NOAA Ocean Exploration,Atlantic Ocean,Deep Scattering Layer
2,EX2107_D01_02W,sample,None,NaN,EX2107_D01_02W,NaN,NaN,NaN,NaN,-77.8525,...,NaN,NaN,NaN,,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_02,NOAA Ocean Exploration,Atlantic Ocean,Benthic Boundary Layer
4,EX2107_D01_06W,sample,None,NaN,EX2107_D01_06W,NaN,NaN,NaN,NaN,-77.8525,...,NaN,NaN,NaN,,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_06,NOAA Ocean Exploration,Atlantic Ocean,Benthic Boundary Layer
6,EX2107_D01_11W,sample,None,NaN,EX2107_D01_11W,NaN,NaN,NaN,NaN,-77.853,...,NaN,NaN,NaN,,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_11,NOAA Ocean Exploration,Atlantic Ocean,Benthic Boundary Layer
8,EX2107_D01_13W,sample,None,NaN,EX2107_D01_13W,NaN,NaN,NaN,NaN,-77.8549,...,NaN,NaN,NaN,Collected at 19:48,NaN,NaN,https://n2t.net/ark:/21547/Fff2EX2107_D01_13,NOAA Ocean Exploration,Atlantic Ocean,Benthic Boundary Layer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,EX2203_PCRBlank_COI,negative control,PCR negative,NaN,not applicable: control sample,NaN,NaN,NaN,NaN,not applicable: control sample,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
973,EX2301_PCRBlank_1_COI,negative control,PCR negative,NaN,not applicable: control sample,NaN,NaN,NaN,NaN,not applicable: control sample,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
974,EX2301_PCRBlank_2_COI,negative control,PCR negative,NaN,not applicable: control sample,NaN,NaN,NaN,NaN,not applicable: control sample,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
975,EX2301_PCRBlank_3_COI,negative control,PCR negative,NaN,not applicable: control sample,NaN,NaN,NaN,NaN,not applicable: control sample,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_samples.to_excel('/Users/luke.thompson/Google Drive/My Drive/NOAA/okex/metadata/Okeanos_eDNA_Sample_Tracking_Merged_Samples.xlsx', index=False)

In [17]:
df_libraries = df_metadata3[['samp_name'] + faire_columns_experimentrun].drop_duplicates()

In [18]:
df_libraries

,samp_name,assay_name,pcr_plate_id,lib_id,seq_run_id,lib_conc,lib_conc_unit,lib_conc_meth,phix_perc,mid_foreward,mid_reverse,filename,filename2,checksum_filename,checksum_filename2,associatedSequences,input_read_count
0,EX2107_D01_01W,Eukarya-18S-V4-Stoeck-Piredda,NaN,18S-EX2107D1-01_S1_L001,OKEX2107part-OKEX2205part_ROV-D2,NaN,NaN,NaN,NaN,NaN,NaN,18S-EX2107D1-01_S1_L001_R1_001.fastq.gz,18S-EX2107D1-01_S1_L001_R2_001.fastq.gz,ed08d22bcf9d76327f2d60c85f93c454,ca881cff4725a51451c9689e6dbe14f2,NaN,NaN
1,EX2107_D01_01W,Metazoa-COI-Leray-Geller,NaN,COI-EX2107D1-01_S51_L001,OKEX2107part-OKEX2205part_ROV-D2,NaN,NaN,NaN,NaN,NaN,NaN,COI-EX2107D1-01_S51_L001_R1_001.fastq.gz,COI-EX2107D1-01_S51_L001_R2_001.fastq.gz,a509ccfef32696dcd6495c56baf95261,fb7d290d0c2df76e578824d6a9e984e4,NaN,NaN
2,EX2107_D01_02W,Eukarya-18S-V4-Stoeck-Piredda,NaN,18S-EX2107D1-02_S2_L001,OKEX2107part-OKEX2205part_ROV-D2,NaN,NaN,NaN,NaN,NaN,NaN,18S-EX2107D1-02_S2_L001_R1_001.fastq.gz,18S-EX2107D1-02_S2_L001_R2_001.fastq.gz,5d5f0fc74288f5048b3111709791dcd3,8cdaaa859f687616c1cdbc6f940064e8,NaN,NaN
3,EX2107_D01_02W,Metazoa-COI-Leray-Geller,NaN,COI-EX2107D1-02_S52_L001,OKEX2107part-OKEX2205part_ROV-D2,NaN,NaN,NaN,NaN,NaN,NaN,COI-EX2107D1-02_S52_L001_R1_001.fastq.gz,COI-EX2107D1-02_S52_L001_R2_001.fastq.gz,180e22035e14011ddb3f4dd7c03742f9,1ce9393336d1a4bafa7ff599add758ea,NaN,NaN
4,EX2107_D01_06W,Eukarya-18S-V4-Stoeck-Piredda,NaN,18S-EX2107D1-06_S3_L001,OKEX2107part-OKEX2205part_ROV-D2,NaN,NaN,NaN,NaN,NaN,NaN,18S-EX2107D1-06_S3_L001_R1_001.fastq.gz,18S-EX2107D1-06_S3_L001_R2_001.fastq.gz,7aea11d84cd0988207fc9c550139f41b,b540dc64df1e353b2bf676e7b1900072,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,EX2203_PCRBlank_COI,Metazoa-COI-Leray-Geller,NaN,EX2203-NegativeControl-COI_S48_L001,OKEX2203_CTD,NaN,NaN,NaN,NaN,NaN,NaN,EX2203-NegativeControl-COI_S48_L001_R1_001.fas...,EX2203-NegativeControl-COI_S48_L001_R2_001.fas...,68eff05234e9196df0e05e8ddb219014,5870dba1b7d6f9434d74e017f8ae36c8,NaN,NaN
973,EX2301_PCRBlank_1_COI,Metazoa-COI-Leray-Geller,NaN,BLK-01_S25_L001,OKEX2301_ROV-D2,NaN,NaN,NaN,NaN,NaN,NaN,BLK-01_S25_L001_R1_001.fastq.gz,BLK-01_S25_L001_R2_001.fastq.gz,3a3580f7373a411cd99d3f6d604d6d76,d57c5e701e4797a2d41e076053c45108,NaN,NaN
974,EX2301_PCRBlank_2_COI,Metazoa-COI-Leray-Geller,NaN,BLK-02_S26_L001,OKEX2301_ROV-D2,NaN,NaN,NaN,NaN,NaN,NaN,BLK-02_S26_L001_R1_001.fastq.gz,BLK-02_S26_L001_R2_001.fastq.gz,3882b35ad415f3186dbda9f127ebc86d,3882b35ad415f3186dbda9f127ebc86d,NaN,NaN
975,EX2301_PCRBlank_3_COI,Metazoa-COI-Leray-Geller,NaN,BLK-03_S51_L001,OKEX2301_ROV-D2,NaN,NaN,NaN,NaN,NaN,NaN,BLK-03_S51_L001_R1_001.fastq.gz,BLK-03_S51_L001_R2_001.fastq.gz,9a7d41e58e77cf96addeaa92f792dfca,61f799dd6e1ec4b94abbec3d130ea81f,NaN,NaN


In [19]:
df_libraries.to_excel('/Users/luke.thompson/Google Drive/My Drive/NOAA/okex/metadata/Okeanos_eDNA_Sample_Tracking_Merged_Libraries.xlsx', index=False)

### Next steps

1. Upload .xlsx to Google Drive
2. Open as Google Sheet
3. Copy and paste into FAIRe Google Sheet